In [1]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from dotenv import load_dotenv
import os
from groq import Groq
# Carregandos os dados da .env
load_dotenv() 

True

In [ ]:
import os

from groq import Groq
# Inicializando o cliente Groq com a chave da API
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)
# Criando uma solicitação de conclusão de chat
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)
# Exibindo o conteúdo da resposta
print(chat_completion.choices[0].message.content)


Fast language models are crucial in natural language processing (NLP) as they enable efficient and effective text analysis, generation, and understanding. The importance of fast language models can be seen in several areas:

1. **Real-time Applications**: Fast language models are essential for real-time applications such as chatbots, virtual assistants, and language translation software. They enable rapid processing and response to user input, ensuring a seamless and interactive experience.
2. **Scalability**: Fast language models can handle large volumes of text data, making them ideal for big data analytics, sentiment analysis, and text classification tasks. This scalability is critical for applications that require processing massive amounts of text data, such as social media monitoring or customer feedback analysis.
3. **Low-Latency**: Fast language models minimize latency, which is the delay between the input and output of a system. Low-latency is vital for applications that requi

In [ ]:
# Groq Agent Class
class Agent:
    # Construtor de classe para inicializar o cliente Groq e o sistema
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})
# Método para adicionar uma mensagem do usuário e obter uma resposta do modelo
    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
# Método para executar a solicitação de conclusão de chat
    def execute(self):
        completion = client.chat.completions.create(
            model="llama-3.3-70b-versatile", messages=self.messages
        )
        return completion.choices[0].message.content

In [ ]:
# Definindo o sistema de prompt para o agente
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE 

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

In [6]:
# ferramenta = são as funções que o agente pode chamar
def calculate(operation: str) -> float:
    return eval(operation)


def get_planet_mass(planet) -> float: # Nessa função eu poderia usar uma API para pegar a massa dos planetas, mas como é um exemplo, vou usar valores fixos
    match planet.lower():
        case "earth":
            return 5.972e24
        case "jupiter":
            return 1.898e27
        case "mars":
            return 6.39e23
        case "mercury":
            return 3.285e23
        case "neptune":
            return 1.024e26
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "venus":
            return 4.867e24
        case _:
            return 0.0

In [ ]:
# Instanciando o agente com o cliente Groq e o sistema de prompt
uncle_sam = Agent(client=client, system=system_prompt)

In [ ]:
# Exemplo de uso do agente para calcular a massa da Terra vezes 5
result = uncle_sam("What is the mass of Earth times 5?")
print(result)

Thought: To find the mass of Earth times 5, I need to first find the mass of Earth and then multiply it by 5.
Action: get_planet_mass: Earth
PAUSE


In [9]:
uncle_sam.messages # Olha o histórico de mensagens do agente

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

In [10]:
result = uncle_sam()
print(result)

In [ ]:
# Exemplo de uso da função get_planet_mass
observation = get_planet_mass("Earth")
print(observation)

5.972e+24


In [ ]:
# Continuando o loop do agente com a observação obtida
next_promt = f"Observation: {observation}"
result = uncle_sam(next_promt)
print(result)

Thought: Now that I have the mass of Earth, I can multiply it by 5 to get the final answer.
Action: calculate: 5.972e+24 * 5
PAUSE


In [13]:
uncle_sam.messages # Olha o histórico de mensagens do agente

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

In [14]:
result = uncle_sam()
print(result)
# O agente pode continuar a conversa, chamando o modelo novamente

In [15]:
observation = calculate("3.285e23 * 5")
print(observation)

1.6425e+24


In [16]:
next_promt = f"Observation: {observation}"
result = uncle_sam(next_promt)
print(result)

Thought: The calculation result is the mass of Earth times 5, so I have my final answer.
Action: None
Answer: The mass of Earth times 5 is 2.986e+25.


In [17]:
uncle_sam.messages # Olha o histórico de mensagens do agente

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

In [18]:
import re # Regex para extrair a ação e o argumento

def agent_loop(max_iterations, query):
    agent = Agent(client, system_prompt)
    tools = ['calculate', 'get_planet_mass']
    next_prompt = query
    i = 0
    while i < max_iterations:
        i+= 1
        result = agent(next_prompt)
        print(result)
        
        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break

agent_loop(max_iterations=10, query="What is the mass of Earth plus the mass of Saturn and all of that times 2?")

Thought: To find the mass of Earth plus the mass of Saturn and all of that times 2, I first need to find the individual masses of Earth and Saturn. Then, I can add these masses together and multiply the result by 2.

Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: Now that I have the mass of Earth, I need to find the mass of Saturn. After that, I can add the two masses together and then multiply the result by 2.

Action: get_planet_mass: Saturn
PAUSE
Observation: 5.683e+26
Thought: I now have the masses of both Earth and Saturn. The next step is to add these two masses together. After adding them, I will then multiply the sum by 2 to get the final result.

Action: calculate: 5.972e24 + 5.683e26
PAUSE
Observation: 5.74272e+26
Thought: I have successfully added the masses of Earth and Saturn. The result is 5.74272e+26 kg. Now, I need to multiply this sum by 2 to get the final answer.

Action: calculate: 5.74272e+26 * 2
PAUSE
Observation: 1.148544e+27
Thought: I have 